# Unlocking all effigie animations

The code looks up all hidden animations and adds them to the script file.
Finally, we generate a VPK that can be loaded in the game.

### Installation

1. Go to [all_animations.zip](resources/all_animations.zip) and click `View Raw` on the page
2. Extract in `[steam folder]\steamapps\common\dota 2 beta\game\`
3. Edit `[dota 2 beta dir]\game\dota\gameinfo.gi` and add `Game all_animations` in the `SearchPaths` section

![Spellicons showcase](resources/animation_install.png "Spellicons showcase")

4. Finally, launch Dota 2 and make some effigies

### Code

To run the code below you need `vdf >= 2.0` and `vpk >= 1.10`. Both are available via `pip`

In [1]:
import re
import vdf
import vpk

vpk_path = "/d/Steam/steamapps/common/dota 2 beta/game/dota/pak01_dir.vpk"
anim_path = "scripts/animation_statues.txt"
out_path = 'resources/animations'

pak1 = vpk.VPK(vpk_path)
anim = vdf.load(pak1.get_file(anim_path), mapper=vdf.VDFDict)

# Fill in missing animations

In [2]:
def get_animations(model_path):
    prefix = '/'.join(model_path.split('/')[:-1])
    animations = []
    for model_path in pak1:
        if model_path.startswith(prefix) and model_path.endswith('vanim_c'):
            name = model_path.split('/')[-1]
            if name[0] != '_' and name != 'bind_pose.vanim_c':
                animations.append(name.split('.')[0])
    return set(animations)

for model_path, statue_animations in anim['DOTAStatueHeroes']['animations'].items():
    if not model_path.startswith("models"):
        continue

    for name in get_animations(model_path):
        if name not in statue_animations:
            statue_animations[name] = '1'

# Generate VPK with updated animation script

In [3]:
!rm -rf ./{out_path}/*
!mkdir -p ./{out_path}/scripts
vdf.dump(anim, open("%s/%s" % (out_path, anim_path), 'wb'), pretty=True)

!mkdir -p ./{out_path}/all_animations/
vpk.new(out_path).save("%s/all_animations/pak01_dir.vpk" % out_path)

!cd {out_path} && zip -r ../all_animations.zip all_animations

  adding: all_animations/ (stored 0%)
  adding: all_animations/pak01_dir.vpk (deflated 86%)
